In [1]:
# Cell 1: Setup
%load_ext autoreload
%autoreload 2

import sys
import gc
import pandas as pd
import numpy as np
from pathlib import Path

# Setup path
sys.path.append(str(Path.cwd().parent))
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils

# Khởi tạo experiment
exp_paths = utils.setup_experiment_folder()
utils.log_experiment_details(exp_path=exp_paths['root'])
# ==============================================================================
# CẤU HÌNH GRID SEARCH
# ==============================================================================
# Giả sử bạn đã có danh sách mRMR đầy đủ 67 đặc trưng đã được xếp hạng
# Nếu chưa có, hãy chạy run_mrmr cho 67 cột trước.
RANKED_FEATURES = config.SELECTED_FEATURES # Giả sử đây là list đã sort theo mRMR

# Định nghĩa các ngưỡng thử nghiệm
L_CANDIDATES = [30, 40, 50, 60]      # Số lượng đầu vào cho AE (L)
N_CANDIDATES = [5, 10, 15, 20]       # Số chiều không gian ẩn (N)
K = 20                               # Số đặc trưng mRMR giữ lại cho RF (Cố định)

results = []

# Cell 2: Load & Scale Data 1 lần duy nhất để tiết kiệm thời gian
print("🚀 Loading data for Grid Search...")
df_train, y_train = preprocessing.load_single_dataset_year('2017', binary_mode=True)
df_test, y_test = preprocessing.load_single_dataset_year('2018', binary_mode=True)

scaler = preprocessing.get_scaler()
X_train_full = scaler.fit_transform(df_train.values)
X_test_full = scaler.transform(df_test.values)

# Cell 3: Vòng lặp Grid Search
for L in L_CANDIDATES:
    for N in N_CANDIDATES:
        print(f"\n╔══════════════════════════════════════════════╗")
        print(f"║ 🔍 TESTING: L={L} (AE Input) | N={N} (Latent) ║")
        print(f"╚══════════════════════════════════════════════╝")
        
        try:
            # 1. Chuẩn bị Feature Indices
            ae_input_indices = [config.SELECTED_FEATURES.index(f) for f in RANKED_FEATURES[:L]]
            rf_direct_indices = [config.SELECTED_FEATURES.index(f) for f in RANKED_FEATURES[:K]]
            
            # 2. Cắt dữ liệu
            X_train_ae = X_train_full[:, ae_input_indices]
            X_test_ae  = X_test_full[:, ae_input_indices]
            
            X_train_direct = X_train_full[:, rf_direct_indices]
            X_test_direct  = X_test_full[:, rf_direct_indices]
            
            # 3. Train AE
            # Thiết kế Hidden Layer linh hoạt: [L -> L*0.75 -> L*0.5 -> N]
            hidden = [int(L*0.75), int(L*0.5)]
            ae_model = autoencoder.DeepAutoencoder(input_dim=L, latent_dim=N, hidden_layers=hidden)
            
            # Train
            # Tạo tên file riêng biệt cho từng cấu hình L và N
            ae_temp_save_path = exp_paths['models'] / f"ae_L{L}_N{N}.pth"

            ae_model = autoencoder.train_ae(ae_model, X_train_ae, save_path=ae_temp_save_path)
            
            # 4. Extract Latent
            Z_train = autoencoder.extract_features(ae_model, X_train_ae)
            Z_test  = autoencoder.extract_features(ae_model, X_test_ae)
            
            # 5. Fusion
            X_train_fusion = np.hstack([X_train_direct, Z_train])
            X_test_fusion  = np.hstack([X_test_direct, Z_test])
            
            # 6. Train & Evaluate RF
            rf_save_path = exp_paths["models"] / "rf_model.joblib"
            rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)
            metrics = evaluation.evaluate_model(rf_model, X_test_fusion, y_test, save_dir=None)
            
            # 7. Lưu kết quả
            res = {
                'L_input': L,
                'N_latent': N,
                'Accuracy': metrics['accuracy'],
                'MCC': metrics['mcc'],
                'Recall_Attack': metrics['report']['Attack']['recall'],
                'Precision_Attack': metrics['report']['Attack']['precision']
            }
            results.append(res)
            print(f"✅ Result: Recall={res['Recall_Attack']:.4f}, MCC={res['MCC']:.4f}")
            
        except Exception as e:
            print(f"❌ Error at L={L}, N={N}: {e}")
            
        finally:
            gc.collect()
            # Giải phóng GPU triệt để
            if 'ae_model' in locals(): del ae_model
            if 'rf_model' in locals(): del rf_model
            
            import torch
            torch.cuda.empty_cache() # Xóa cache GPU
            gc.collect()             # Dọn dẹp RAM

# Cell 4: Phân tích kết quả
df_results = pd.DataFrame(results)
df_results.to_csv(exp_paths['root'] / "grid_search_results.csv", index=False)

# Tìm "Điểm ngọt" (Ưu tiên Recall_Attack cao nhất, sau đó đến MCC)
sweet_spot = df_results.sort_values(by=['Recall_Attack', 'MCC'], ascending=False).iloc[0]

print("\n🏆 THE SWEET SPOT FOUND:")
print(sweet_spot)

📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251218_184735
       EXPERIMENT SUMMARY - 2025-12-18 18:47:35
🚀 Device: cuda
📂 Exp Path: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251218_184735

--- DATA CONFIGURATION ---
🔹 Binary Mode: True
🔹 Random Seed: 42
🔹 Selected Features (Total): 65
🔹 mRMR Features (K): 20

--- AUTOENCODER HYPERPARAMETERS ---
🔸 Input Dim: 40
🔸 Latent Dim (N): 20
🔸 Hidden Layers: [30, 20]
🔸 Epochs: 20
🔸 Batch Size: 16384
🔸 Learning Rate: 0.001

--- RANDOM FOREST HYPERPARAMETERS ---
🔸 Estimators: 100
🔸 Max Depth: 12
✅ Experiment details saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251218_184735\experiment_details.txt
🚀 Loading data for Grid Search...
🔄 Loading dataset year 2017 (Binary=True)...
✅ Loaded 2017. Shape: (2830743, 65)
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)

╔══════════════════════════════════════════════╗
║ 

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.3min


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

...

building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  5.5min finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251218_184735\models\rf_model.joblib

📊 Evaluating on Test Set...


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   12.2s finished


   ✅ Accuracy: 0.7619
   ⭐ MCC:      0.3353
✅ Result: Recall=0.2810, MCC=0.3353

🏆 THE SWEET SPOT FOUND:
L_input             30.000000
N_latent             5.000000
Accuracy             0.771801
MCC                  0.381554
Recall_Attack        0.386792
Precision_Attack     0.675257
Name: 0, dtype: float64
